<a href="https://colab.research.google.com/github/currencyfxjle/KNAPSACK_GDF_5.0/blob/main/KNAPSACK_GDF_5_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread
!pip install gspread-dataframe
!pip install --upgrade oauth2client
!pip install xlsxwriter

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

# List files and folders under My Drive
base_path = '/content/drive/My Drive'
for root, dirs, files in os.walk(base_path):
    print(f"Directory: {root}")
    for directory in dirs:
        print(f"  Sub-directory: {directory}")
    for file in files:
        print(f"  File: {file}")

import os

correct_path = '/content/drive/My Drive/Colab Notebooks/GDF - Knapsack/2025 Project Agenda.gsheet'

if os.path.exists(correct_path):
    print("File exists:", correct_path)
else:
    print("File does NOT exist:", correct_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00
Mounted at /content/drive
Directory: /content/drive/My Drive
  Sub-directory: Risk Modeling
  Sub-directory: Colab Notebooks
  File: Financial Concepts   Solar Energy Greenday Finance.gdoc
  File: config.zip
  File: 2024 Project Allocation Results (7).gsheet
  File: 2024 Project Allocation Results (6).gsheet
  File: 2024 Project Allocation Results (5).gsheet
  File: 2024 Project Allocation Results (4).gsheet
  File: 2024 Project Allocation Results (3).gsheet
  File: 2024 Project Allocation Results (2).gsheet
  File: 2024 Project Allocation Results (1).gsheet
  File: 2024 Project Allocation Results.gsheet
  File: Estados de Resultados - Victor Lizarraga.gdoc
  File: Ajustes  knapsack Lalo.gsheet
  File: Backup 2025.gsheet
  File: Ruta calculadora.gdoc
  File: KNAPSACK - GDF - Last Version 2.1.ipynb
  File: Knapsack Project.gdoc
Directory: /content/drive/My Drive/Risk Modeling
  File: Risk Model - Volatility 

In [7]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']  # Only consider projects with 'Y'
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Clean and convert GDF Profit (remove '%' and convert to numeric)
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100  # Convert percentage to decimal

    supply_df = supply_df.dropna()

    # Sort projects by Poject_date (ascending) and GDF Profit (descending)
    supply_df = supply_df.sort_values(
        by=['Poject_date', 'GDF Profit'],
        ascending=[True, False]
    )

    # Save original Est Cash Amount and add a "Fully Allocated" flag
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Categorize Est Cash Amount into buckets
    supply_df['Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )
    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['historic allocated amount'] = pd.to_numeric(demand_df['allocated_amount'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['original_pending_amount'] = demand_df['updated pending amount']  # Save original pending amount
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Categorize Total Appetite into buckets
    demand_df['Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )
    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    total_allocated = 0
    allocations = []
    unallocated_demand = []
    project_tracking = []

    # Sort purchasers by approximated_appetite (descending)
    demand_df = demand_df.sort_values(
        by='approximated_appetite',
        ascending=False
    )

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        remaining_appetite = purchaser['remaining_appetite']  # Track remaining appetite
        effective_date = purchaser['effective_date_purchaser']

        # Iterate through projects (sorted by date and profit)
        for _, project in supply_df.iterrows():
            if project['Fully Allocated']:
                continue  # Skip already allocated projects

            project_cost = project['Est Cash Amount']
            project_name = project['Proyect Name']
            project_id = project['Project ID']
            project_date = project['Poject_date']

            # Check if the project can be fully acquired and respects the date constraint
            if (project_cost <= remaining_pending_amount and
                project_cost <= remaining_appetite and  # Ensure it fits within remaining appetite
                project_date >= effective_date):
                # Allocate the project
                allocations.append({
                    "Purchaser": purchaser['purchaser_name'],
                    "Project ID": project_id,
                    "Project Name": project_name,
                    "Original Est Cash Amount": project['Original Est Cash Amount'],
                    "Allocated Amount": project_cost,
                    "Total Appetite": purchaser['approximated_appetite'],  # Original total appetite
                    "Remaining Appetite": remaining_appetite - project_cost,  # Updated remaining appetite
                    "Updated Pending Amount": remaining_pending_amount - project_cost,
                    "Effective Date Purchaser": effective_date,
                    "Project Date": project_date.date(),
                    "Fully Allocated": "Yes",
                    "Purchaser Category": purchaser['Category'],
                    "Project Category": project['Category'],
                    "GDF Profit": project['GDF Profit']
                })
                total_allocated += project_cost
                remaining_pending_amount -= project_cost
                remaining_appetite -= project_cost  # Reduce remaining appetite
                supply_df.loc[project.name, 'Fully Allocated'] = True  # Mark project as fully allocated

                # Track project allocation
                project_tracking.append({
                    "Project ID": project_id,  # Include Project ID
                    "Project Name": project_name,
                    "Total Chunks": 1,  # Assuming each project is a single chunk
                    "Chunks Acquired": 1,
                    "Chunks Remaining": 0,
                    "Acquired Amounts": project_cost,
                    "Acquired By": purchaser['purchaser_name'],
                    "Total Appetite": purchaser['approximated_appetite'],  # Original total appetite
                    "Updated Pending Amount": remaining_pending_amount,
                    "Project Category": project['Category'],
                    "GDF Profit": project['GDF Profit']
                })

                if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                    break  # Move to the next purchaser

        # Update the purchaser's remaining pending amount and remaining appetite
        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount
        demand_df.at[idx, 'remaining_appetite'] = remaining_appetite  # Track remaining appetite

        # Track unallocated demand
        if remaining_pending_amount > 0 or remaining_appetite > 0:
            unallocated_demand.append({
                "Purchaser Name": purchaser['purchaser_name'],
                "Approximated Appetite": purchaser['approximated_appetite'],
                "Historic Allocated Amount": purchaser['historic allocated amount'],
                "Updated Pending Amount": remaining_pending_amount,
                "Remaining Appetite": remaining_appetite,  # Include remaining appetite
                "Deal Stage": purchaser['deal_stage'],
                "State": purchaser['state'],
                "Effective Date Purchaser": effective_date
            })

    return total_allocated, allocations, unallocated_demand, supply_df, project_tracking

# Iterative Allocation Process
def iterative_allocation(supply_df, demand_df, max_iterations=5):
    iteration = 0
    total_allocated = 0
    all_allocations = []
    project_tracking = []

    while iteration < max_iterations:
        iteration += 1
        current_allocated, allocations, unallocated_demand, remaining_supply, tracking = match_supply_to_demand(supply_df, demand_df)
        total_allocated += current_allocated
        all_allocations.extend(allocations)
        project_tracking.extend(tracking)

        if current_allocated == 0:
            break

        supply_df = remaining_supply[remaining_supply['Est Cash Amount'] > 0].copy()

    return total_allocated, all_allocations, supply_df, demand_df, project_tracking

# Save Results to Separate Tabs in Excel
def save_to_excel(allocations, summary, unallocated_demand, remaining_supply, project_tracking, filename):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
        pd.DataFrame(summary).to_excel(writer, sheet_name="Summary", index=False)
        pd.DataFrame(unallocated_demand).to_excel(writer, sheet_name="Unallocated Demand", index=False)
        remaining_supply.to_excel(writer, sheet_name="Remaining Supply", index=False)
        pd.DataFrame(project_tracking).to_excel(writer, sheet_name="Project Tracking", index=False)  # Keep detailed rows

    print(f"Results saved to: {filename}")

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()
total_allocated, allocations, remaining_supply, remaining_demand, project_tracking = iterative_allocation(supply_df, demand_df)

# Calculations for Summary
total_project_value = supply_df['Original Est Cash Amount'].sum()

# Calculate Remaining Supply Value After: Sum of Est Cash Amount for projects with Fully Allocated = False
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

# Original Total Unallocated Value (Total Appetite)
total_unallocated_value_appetite = demand_df['approximated_appetite'].sum()

# Total Allocated: Sum of Allocated Amount in allocations
total_allocated = sum(project['Allocated Amount'] for project in allocations)

# Total Pending Amount: Original Total Unallocated Value (Total Appetite) minus Total Allocated
total_pending_amount = total_unallocated_value_appetite - total_allocated

# Validation: Ensure Acquired Amounts match Allocated Amounts
total_acquired_amounts = sum(project['Acquired Amounts'] for project in project_tracking)
if total_allocated == total_acquired_amounts:
    print("Validation Passed: Acquired Amounts match Allocated Amounts.")
else:
    print(f"Validation Failed: Acquired Amounts ({total_acquired_amounts}) do not match Allocated Amounts ({total_allocated}).")

# Summary Data
summary_data = pd.DataFrame([
    {"Metric": "Original Total Unallocated Value (Total Appetite)", "Value": total_unallocated_value_appetite},
    {"Metric": "Original Total Project Value (Est Cash Amount)", "Value": total_project_value},
    {"Metric": "Total Allocated", "Value": total_allocated},
    {"Metric": "Remaining Supply Value After", "Value": remaining_supply_value},
    {"Metric": "Total Pending Amount", "Value": total_pending_amount},
])

# Save Results to Excel with Tabs
save_to_excel(
    allocations,
    summary_data,
    remaining_demand,
    remaining_supply,
    project_tracking,
    "allocation_results.xlsx"
)

# Results Summary
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {total_unallocated_value_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {total_project_value}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

Validation Passed: Acquired Amounts match Allocated Amounts.
Results saved to: allocation_results.xlsx

Results Summary:
Original Total Unallocated Value (Total Appetite): 110880429.1
Original Total Project Value (Est Cash Amount): 44025757.750000015
Total Allocated: 44025757.75000002
Remaining Supply Value After: 0.0
Total Pending Amount: 66854671.34999997


In [8]:
# Match Supply to Demand (with detailed logging and tracking)
def match_supply_to_demand(supply_df, demand_df):
    total_allocated = 0
    allocations = []
    unallocated_demand = []
    project_tracking = []
    valid_count = 0  # Track valid allocations
    invalid_count = 0  # Track invalid allocations

    # Sort purchasers by approximated_appetite (descending)
    demand_df = demand_df.sort_values(
        by='approximated_appetite',
        ascending=False
    )

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        remaining_appetite = purchaser['remaining_appetite']
        effective_date = purchaser['effective_date_purchaser']

        print(f"\nProcessing Purchaser: {purchaser['purchaser_name']}")
        print(f"Remaining Pending Amount: {remaining_pending_amount}")
        print(f"Remaining Appetite: {remaining_appetite}")
        print(f"Effective Date: {effective_date}")

        # Iterate through projects (sorted by date and profit)
        for _, project in supply_df.iterrows():
            if project['Fully Allocated']:
                continue  # Skip already allocated projects

            project_cost = project['Est Cash Amount']
            project_name = project['Proyect Name']
            project_id = project['Project ID']
            project_date = project['Poject_date']

            # Check constraints
            valid = True
            reason = []

            if project_cost > remaining_pending_amount:
                valid = False
                reason.append("Exceeds Remaining Pending Amount")
            if project_cost > remaining_appetite:
                valid = False
                reason.append("Exceeds Remaining Appetite")
            if project_date < effective_date:
                valid = False
                reason.append("Project Date Before Effective Date")

            if valid:
                # Allocate the project
                allocations.append({
                    "Purchaser": purchaser['purchaser_name'],
                    "Project ID": project_id,
                    "Project Name": project_name,
                    "Original Est Cash Amount": project['Original Est Cash Amount'],
                    "Allocated Amount": project_cost,
                    "Total Appetite": purchaser['approximated_appetite'],
                    "Remaining Appetite": remaining_appetite - project_cost,
                    "Updated Pending Amount": remaining_pending_amount - project_cost,
                    "Effective Date Purchaser": effective_date,
                    "Project Date": project_date.date(),
                    "Fully Allocated": "Yes",
                    "Purchaser Category": purchaser['Category'],
                    "Project Category": project['Category'],
                    "GDF Profit": project['GDF Profit']
                })
                total_allocated += project_cost
                remaining_pending_amount -= project_cost
                remaining_appetite -= project_cost
                supply_df.loc[project.name, 'Fully Allocated'] = True

                # Track project allocation
                project_tracking.append({
                    "Project ID": project_id,
                    "Project Name": project_name,
                    "Total Chunks": 1,
                    "Chunks Acquired": 1,
                    "Chunks Remaining": 0,
                    "Acquired Amounts": project_cost,
                    "Acquired By": purchaser['purchaser_name'],
                    "Total Appetite": purchaser['approximated_appetite'],
                    "Updated Pending Amount": remaining_pending_amount,
                    "Project Category": project['Category'],
                    "GDF Profit": project['GDF Profit']
                })

                valid_count += 1  # Increment valid allocation counter
                print(f"✅ Valid Allocation: Project '{project_name}' (ID: {project_id}) allocated to {purchaser['purchaser_name']}")
                print(f"  Allocated Amount: {project_cost}")
                print(f"  Updated Remaining Pending Amount: {remaining_pending_amount}")
                print(f"  Updated Remaining Appetite: {remaining_appetite}")

            else:
                invalid_count += 1  # Increment invalid allocation counter
                print(f"❌ Invalid Allocation: Project '{project_name}' (ID: {project_id}) not allocated to {purchaser['purchaser_name']}")
                print(f"  Reasons: {', '.join(reason)}")

            if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                break  # Move to the next purchaser

        # Update the purchaser's remaining pending amount and remaining appetite
        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount
        demand_df.at[idx, 'remaining_appetite'] = remaining_appetite

        # Track unallocated demand
        if remaining_pending_amount > 0 or remaining_appetite > 0:
            unallocated_demand.append({
                "Purchaser Name": purchaser['purchaser_name'],
                "Approximated Appetite": purchaser['approximated_appetite'],
                "Historic Allocated Amount": purchaser['historic allocated amount'],
                "Updated Pending Amount": remaining_pending_amount,
                "Remaining Appetite": remaining_appetite,
                "Deal Stage": purchaser['deal_stage'],
                "State": purchaser['state'],
                "Effective Date Purchaser": effective_date
            })

    return total_allocated, allocations, unallocated_demand, supply_df, project_tracking, valid_count, invalid_count

# Iterative Allocation Process
def iterative_allocation(supply_df, demand_df, max_iterations=5):
    iteration = 0
    total_allocated = 0
    all_allocations = []
    project_tracking = []
    total_valid = 0  # Track total valid allocations
    total_invalid = 0  # Track total invalid allocations

    while iteration < max_iterations:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        current_allocated, allocations, unallocated_demand, remaining_supply, tracking, valid_count, invalid_count = match_supply_to_demand(supply_df, demand_df)
        total_allocated += current_allocated
        all_allocations.extend(allocations)
        project_tracking.extend(tracking)
        total_valid += valid_count  # Accumulate valid allocations
        total_invalid += invalid_count  # Accumulate invalid allocations

        if current_allocated == 0:
            break

        supply_df = remaining_supply[remaining_supply['Est Cash Amount'] > 0].copy()

    print(f"\nTotal Iterations: {iteration}")
    print(f"Total Valid Allocations: {total_valid}")
    print(f"Total Invalid Allocations: {total_invalid}")

    return total_allocated, all_allocations, remaining_supply, remaining_demand, project_tracking

# Save Results to Separate Tabs in Excel
def save_to_excel(allocations, summary, unallocated_demand, remaining_supply, project_tracking, filename):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
        pd.DataFrame(summary).to_excel(writer, sheet_name="Summary", index=False)
        pd.DataFrame(unallocated_demand).to_excel(writer, sheet_name="Unallocated Demand", index=False)
        remaining_supply.to_excel(writer, sheet_name="Remaining Supply", index=False)
        pd.DataFrame(project_tracking).to_excel(writer, sheet_name="Project Tracking", index=False)

    print(f"Results saved to: {filename}")

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()
total_allocated, allocations, remaining_supply, remaining_demand, project_tracking = iterative_allocation(supply_df, demand_df)

# Calculations for Summary
total_project_value = supply_df['Original Est Cash Amount'].sum()
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()
total_unallocated_value_appetite = demand_df['approximated_appetite'].sum()
total_allocated = sum(project['Allocated Amount'] for project in allocations)
total_pending_amount = total_unallocated_value_appetite - total_allocated

# Validation: Ensure Acquired Amounts match Allocated Amounts
total_acquired_amounts = sum(project['Acquired Amounts'] for project in project_tracking)
if total_allocated == total_acquired_amounts:
    print("\nValidation Passed: Acquired Amounts match Allocated Amounts.")
else:
    print(f"\nValidation Failed: Acquired Amounts ({total_acquired_amounts}) do not match Allocated Amounts ({total_allocated}).")

# Summary Data
summary_data = pd.DataFrame([
    {"Metric": "Original Total Unallocated Value (Total Appetite)", "Value": total_unallocated_value_appetite},
    {"Metric": "Original Total Project Value (Est Cash Amount)", "Value": total_project_value},
    {"Metric": "Total Allocated", "Value": total_allocated},
    {"Metric": "Remaining Supply Value After", "Value": remaining_supply_value},
    {"Metric": "Total Pending Amount", "Value": total_pending_amount},
])

# Save Results to Excel with Tabs
save_to_excel(
    allocations,
    summary_data,
    remaining_demand,
    remaining_supply,
    project_tracking,
    "allocation_results.xlsx"
)

# Results Summary
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {total_unallocated_value_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {total_project_value}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
❌ Invalid Allocation: Project 'Lewistone Solar II' (ID: Lewistone Solar II34) not allocated to Jonathan Hagen
  Reasons: Exceeds Remaining Pending Amount, Exceeds Remaining Appetite, Project Date Before Effective Date
❌ Invalid Allocation: Project 'Lewistone Solar II' (ID: Lewistone Solar II35) not allocated to Jonathan Hagen
  Reasons: Exceeds Remaining Pending Amount, Exceeds Remaining Appetite, Project Date Before Effective Date
❌ Invalid Allocation: Project 'Lewistone Solar II' (ID: Lewistone Solar II36) not allocated to Jonathan Hagen
  Reasons: Exceeds Remaining Pending Amount, Exceeds Remaining Appetite, Project Date Before Effective Date
❌ Invalid Allocation: Project 'Lewistone Solar II' (ID: Lewistone Solar II37) not allocated to Jonathan Hagen
  Reasons: Exceeds Remaining Pending Amount, Exceeds Remaining Appetite, Project Date Before Effective Date
❌ Invalid Allocation: Project 'Lewistone Solar II' (ID: Lewis